# Segmenting & Clustering Toronto Neighborhoods
## Installing Modules

In [2]:
!pip install geocoder

     |████████████████████████████████| 102kB 12.6MB/s ta 0:00:01


In [3]:
!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Using Geolocator

In [4]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")

location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


In [5]:
address = 'M1C'

geolocator = Nominatim(user_agent="ca_explorer")

location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 50.4215446, 14.9213332628019.


I decided that I'd rather use the csv file instead. 

## Creating a Dataframe from the CSV file

In [6]:
import pandas as pd 
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')
print(df_coord.shape)
df_coord.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Converting "Postal Code" to "Postcode" for consistency
This is so this dataframe can be merged with the scraped wiki dataframe later

In [7]:
df_coord = df_coord.rename(columns={'Postal Code': 'Postcode'}) 
df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Recreating the table of scraped web data

In [11]:
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)
df=tables[0]

borough_mask = df.index[df['Borough'] == 'Not assigned']
neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
neighborhood_and_borough_mask = borough_mask & neighborhood_mask

df.drop(df.index[borough_mask], inplace=True)
df.reset_index(drop=True, inplace=True)

neighborhood_mask = df.index[df['Neighbourhood'] == 'Not assigned']
for idx in neighborhood_mask:
    df['Neighbourhood'][idx] = df['Borough'][idx]
    

df2 = df.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
df_grouped=df2.reset_index()

df_grouped.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Grouping Tables

In [12]:
df_final = df_grouped.groupby(['Postcode','Borough'], sort=False).agg(lambda x: ','.join(x))
df_final.reset_index(level=['Postcode','Borough'], inplace=True)
print(df_final.shape)
df_final.head(10)

(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Merging Tables to create final Dataframe with Latitude and Longitude

In [13]:
df_final_2 = pd.merge(df_final, df_coord, on='Postcode')
print(df_final_2.shape)
df_final_2.head(10)

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
